In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#Definicion constantes
N=1000 #numero de agentes
M=1000 #dinero total
W=15 #numero de estados
nsteps=100000 #pasos de tiempo
#random.seed(1)
T=np.divide(M,N)#dinero promedio
agents=np.zeros(N) #creamos un array en cada casilla tenemos el dinero del agente i-esimo
credito=np.zeros(N)

In [3]:
#1. funcion de inicializacion 
def initialize_agents(ag,size,prom):
    for i in range(size):
        ag[i]=prom 

In [4]:
#funcion 2 de inicializacion
def initialize2(ag,size,prom,ndif):
    for s in range(ndif):
        ag[s]=0.0
    for l in range(ndif,size,1):
        ag[l]=np.divide((size*prom),(size-ndif))

In [5]:
def interaction(ag,N,nsteps,Lambda,cr):
    #epsilon=random.uniform(-1,1) #primer cambio epsilon entre -1 y 1
    mdeuda=-1*2
    l_entropia=[]
    for k in range(nsteps):
        epsilon=random.uniform(-1,1)
        i=random.randint(0,N-1)
        j=random.randint(0,N-1) 
        if i!=j:
            '''
            print('i',i)
            print('j',j)
            print('vector credito antes de interactuar', cr)
            '''
            tempi=ag[i]
            tempj=ag[j]
            delta_m=(1-Lambda)*(epsilon*tempj-(1-epsilon)*tempi)
            '''
            print('tempi',tempi)
            print('tempj', tempj)
            print('deltam',delta_m)
            '''
            mi = tempi + delta_m #winner
            mj = tempj - delta_m #loser
            '''
            print('mi despues de interactuar', mi)
            print('mj despues de interactuar', mj)
            '''
            crfi = cr[i]-np.abs(mi)#vi' (si i perdiera y se hiciera la transaccción)      
            crfj = cr[j]-np.abs(mj)# vj' (si j perdiera y se hiciera la transaccción)
            #condicion para j  -> deuda cuando pido dinero prestado
            if(delta_m>=0):
                if ((crfj>mdeuda and mj>mdeuda) and (mj<0 and mi>=np.abs(mj))): #segundo cambio agrupar los ands
                     #tercer cambio poner if en otra posicion
                    cr[j] += mj        #vector v para guardar la deuda 
                       #condicion para i  -> credito cuando presto dinero
                    cr[i] += -1*mj #para garantizar que i tenga dinero suficiente para hacer el prestamo
                    mi=mi-np.abs(mj)
                    mj=0
                    '''    
                    print('credito j',cr[j])
                    print('credito i', cr[i])
                    print('mi despues credito', mi)
                    print('mj despues credito', mj)
                    '''
                #print('vector credito', cr)
                if ((mi<np.abs(mj)) or (mj<mdeuda)): #no se hace nada si i no le ṕuede prestar
                    mj=tempj
                    mi=tempi
            if(delta_m<0): #si j es el que presta
                if ((crfi>mdeuda and mi>mdeuda) and (mi<0 and mj>=np.abs(mi))): #segundo cambio agrupar los ands
                #tercer cambio poner if en otra posicion
                    cr[i] += mi        #vector v para guardar la deuda 
                #condicion para i  -> credito cuando presto dinero
                    cr[j] += -1*mi #para garantizar que i tenga dinero suficiente para hacer el prestamo
                    mj=mj-np.abs(mi)
                    mi=0
                    '''
                    print('credito i',cr[i])
                    print('credito j', cr[j])
                    print('mi despues credito', mi)
                    print('mj despues credito', mj)
                    '''    
                #print('vector credito', cr)
                if ((mj<np.abs(mi)) or (mi<mdeuda)):
                    mi=tempi
                    mj=tempj
            #guardar los ingresos en u
            ag[i]=mi
            ag[j]=mj
            '''
            print('ingreso i', ag[i])
            print('ingreso j', ag[j])
            print('fin del paso de tiempo')
            print('         ')
            '''
            #u viene de los ingresos u=[0,infinto] y v=[-mdeuda,infinito]


In [6]:
def entropia(ag,w):
    #parametros para los intervalos
    S=ag.size # numero de agentes
    Imin=np.min(ag)
    Imax=np.max(ag)
    ancho=np.divide((Imax-Imin),1.0*w)
    intervalos=[]
    #parametros calculo de la entropia de Shannon
    pi=np.zeros(w)
    pilnpi=np.zeros(w)
    frec=np.zeros(w)
    #crear los intervalos
    for k in range(w):
        mink=Imin+k*ancho
        maxk=Imin+(k+1)*ancho
        intervalos.append(mink)
        #empezamos a calcular las frecuencias
        if k==w-1: #para el ultimo intervalo la condicion debe ser cerrada para maxk
            frec[k]=len(ag[(ag>=mink) & (ag<=maxk)])
        else:
            frec[k]=len(ag[(ag>=mink) & (ag<maxk)])
        #una vez que ya tenemos la frecuencia, calculamos pi
        pi[k]=np.divide(frec[k],1.0*S)
        #calculamos pi*lnpi
        if pi[k]==0:
            pilnpi[k]=0
        else:
            pilnpi[k]=pi[k]*np.log(pi[k])
    #agregamos el ultimo limite a los intervalos
    intervalos.append(Imax)
    #calculamos la suma de pilnpi
    sumpilnpi=np.sum(pilnpi)
    entropy=-1*sumpilnpi
    #al final retornamos los intervalos, las frecuencias y la entropia
    return intervalos,frec,entropy


In [7]:
#funcion para calcular el gini
def gini(df0):
    N=df0.size
    Itotal=np.sum(df0)
    #creamos arrays para calcular poblacion e ingreso acumulado
    Xing=np.zeros(N) #ingreso de cada individuo/ingreso total
    Xcum=np.zeros(N) #ingreso acumulado
    Xsum=np.zeros(N) #Xk+Xk-1
    Ypob=np.zeros(N) #1/pobtotal
    Ycum=np.zeros(N) #poblacion agregada
    Ydiff=np.zeros(N) #Yk-Yk-1
    XsumYdiff=np.zeros(N) #(Xk+Xk-1)(Yk-Yk-1)
    
    for i in range(N):
        #llenamos array Xing
        Xing[i]=1.0*df0[i]/Itotal
        #llenamos array Xcum
        if i==0:
            Xcum[i]=Xing[i]
        else:
            Xcum[i]=Xing[i]+Xcum[i-1]
        #llenamos array Xsum
        if i==0:
            Xsum[i]=Xcum[i]
        else:
            Xsum[i]=Xcum[i]+Xcum[i-1]
        #hacemos lo mismo para Y
        Ypob[i]=1.0/N
        if i==0:
            Ycum[i]=Ypob[i]
        else:
            Ycum[i]=Ypob[i]+Ycum[i-1]
        if i==0:
            Ydiff[i]=Ycum[i]
        else:
            Ydiff[i]=Ycum[i]-Ycum[i-1]
        #llenar array de multiplicacion (Xk+Xk+1)(Yk-Yk-1)
        XsumYdiff[i]=Xsum[i]*Ydiff[i]
    #una vez todo este lleno calculamos la suma
    sumaXsumYdiff=np.sum(XsumYdiff)
    Gini=1.0-sumaXsumYdiff
    return Xcum, Ycum, Gini

In [8]:
#funcion para calcular el Theil
def theil(df0):
    N=df0.size
    Itotal=np.sum(df0)
    Ui=np.zeros(N) #array fraccion de ingreso con respecto al total
    UilnUi=np.zeros(N)
    for j in range(N):
        Ui[j]=1.0*df0[j]/Itotal
        if Ui[j]==0:
            UilnUi[j]=0
        else:
            UilnUi[j]=Ui[j]*np.log(Ui[j])
    #calculamos la suma Ui*lnUi
    sumaUilnUi=np.sum(UilnUi)
    Theil=1+sumaUilnUi
    Theil_norm=1+np.divide(sumaUilnUi,np.log(N))
    return Theil_norm

In [9]:
#Movilidad Social 
def movilidad(final, inicial, numAgentes):
    diferencia = final-inicial
    '''
    a=[1,2,3]
    a>1
    b=[False, True, True]
    c=a[a>1]
    c=[2,3]  
    '''
    #print('diferencia', diferencia)
    num = len(diferencia[diferencia>0])
    print('# de agentes que aumentaron su ingreso', num)
    probabilidad = num/numAgentes
    print('probabilidad de aumentar ingreso', probabilidad)
    return probabilidad

In [10]:
#calcular vector de ingresos promedio al repetir varias veces la simulacion

nsim=1
Lambda=0.7
suma_total=np.zeros(N)
agentesprom=np.zeros([nsim,N])#creamos una matriz con nsim filas y N columnas
ingresoprom=np.zeros([nsim,N])
#agentesprom=np.zeros(N)
l_entroprom=[]

for i in range(nsim):
    initialize_agents(agents,N,T)
    #l_entropia=interaction(agents,N,nsteps,W)
    interaction(agents,N,nsteps,Lambda,credito)
    for l in range(N):
        suma_total[l]=agents[l]+credito[l]
        
    #hacer el u+v
    #print(agents)
    #print('-----------------------')
    #print(credito)
    #print('fin de la simulacion')
    #ordenar la suma
    ing_sort=np.sort(agents)
    ingresoprom[i]=ing_sort
    ag_sort=np.sort(suma_total)
    agentesprom[i]=ag_sort
    agents=np.zeros(N)
    credito=np.zeros(N)

[ 6.73526006e+00 -1.62188098e+00 -1.72181288e+00  8.16441362e-01
 -1.04660349e+00 -1.92883839e+00  3.29323369e+00  1.19668521e+01
 -1.36624878e+00  1.96706796e+00 -1.57953507e+00  1.16018569e+01
  7.52279002e-01 -1.76225068e+00  0.00000000e+00  2.97098180e+00
  8.52282109e+00  0.00000000e+00  9.42196901e+00 -1.73923353e+00
 -1.43350828e+00  2.43652355e+01 -1.28527678e+00 -1.89731792e+00
 -1.96224847e+00 -1.56031433e+00 -1.92242081e+00 -1.49932337e+00
 -1.49738521e+00  0.00000000e+00  4.02238066e-01  0.00000000e+00
 -1.69711471e+00  7.38930071e+00 -1.65732959e+00  0.00000000e+00
 -1.36985699e+00  0.00000000e+00 -1.73067354e+00 -1.98442499e+00
  2.43006893e+00 -1.86041256e+00 -1.74339411e+00  2.59125490e+01
 -1.68080905e+00  6.82916046e+00  0.00000000e+00  0.00000000e+00
  1.83003256e+01 -1.72998523e+00 -1.45762189e+00 -1.92058871e+00
 -1.79592158e+00 -2.94151081e-01  2.54952850e+01  1.99472724e+00
  7.41030642e+00  6.00408560e+00  3.64811859e+00 -1.51446326e+00
  2.24054402e+00 -1.49305

In [ ]:
#promedio de m
promedio_ag=np.mean(agentesprom,axis=0)

In [ ]:
#calculo de la movilidad del promedio
movilidad(promedio_ag, T, N)

In [ ]:
#promedio de los ingresos u
promedio_ing=np.mean(ingresoprom,axis=0)


In [ ]:
intervals, frecuencies, s_eq=entropia(promedio_ag,W)

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.hist(x=promedio_ag,bins=intervals,color="skyblue",rwidth=0.85,lw=1,edgecolor='black')
plt.xlabel('Rangos por ingreso')
plt.ylabel('Distribucion de individuos')
#plt.xticks(intervals)
#fig.savefig('CC01.png')

In [ ]:
intervalos_ing, frecuencias_ing, s_ing=entropia(promedio_ing,W)

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.hist(x=promedio_ing,bins=intervalos_ing,color="skyblue",rwidth=0.85,lw=1,edgecolor='black')
plt.xlabel('Rangos por ingreso')
plt.ylabel('Distribucion de ingreso de los individuos')
#plt.xticks(intervals)
#fig.savefig('CC01.png')

In [ ]:
print('Intervalos ingreso', intervalos_ing)
print('Numero de ocupacion por clase', frecuencias_ing)

In [ ]:
#movilidad social segun el articulo
def movilidad_art(bins_ing,frec_ing,W):
    prob_enc=np.zeros(W)
    r=bins_ing
    r_prom=np.zeros(W)
    pkXk=np.zeros(W)
    i=3
    sum_k=0.0
    mob=0.0
    r_n=np.divide(r[W-2]+r[W-1],2)
    r_iprom=np.divide(r[i+1]+r[i],2)
    for k in range(1,W,1):
        r_prom[k]=np.divide((r[k-1]+r[k]),2)
        r_prom[i]=np.divide((r[i-1]+r[i]),2)
        prob_enc[k]=np.divide(min(r_prom[k],r_prom[i]),4*r_n)
        print(prob_enc[k])
        pkXk[k]=prob_enc[k]*frec_ing[k]
    sum_k=np.sum(pkXk)
    mob=np.divide(sum_k,(r[i+1]-r[i]))
    return mob


In [ ]:
movilidad_art(intervalos_ing,frecuencias_ing,W)


In [ ]:
print(np.sum(promedio_ag))

In [ ]:
#enviar datos a excel
df1=pd.DataFrame(data=promedio_ag)
df1.to_excel('promedioagentesCC07.xlsx')

In [ ]:
print("Intervalos :", intervals)
print("ni por rango:", frecuencies)

In [ ]:
#enviar datos a excel
IntervalosCC07=pd.DataFrame(data=intervals)
FrecCC07=pd.DataFrame(data=frecuencies)
IntervalosCC07.to_excel('intervalosCC07.xlsx')
FrecCC07.to_excel('frecuCC07.xlsx')

In [ ]:
#Gini
#Primero Verificamos que nuestro vector de dinero promedio este ordenado
np.all(np.diff(promedio_ing) >= 0)


In [ ]:
#Calculamos el Gini
lista1Xcum, lista1Ycum, l1gini = gini(promedio_ing)

print('El índice de Gini: ')
print(l1gini)
#print(np.sum(promedio_ag))

In [ ]:
#Theil 
print('El índice de Theil: ')
l1theil=theil(promedio_ag)
print(l1theil)

In [ ]:
#La entropia de Shannon para m=W
print('La entropia de Shannon para m=W: ')
inter,l1frecW,l1entrW = entropia(promedio_ag,W)
print(l1entrW)

In [ ]:
#La entropia de Shannon para m=20
print('La entropia de Shannon para m=20: ')
inter20,l1frec20,l1entr20 = entropia(promedio_ag,20)
print(l1entr20)
#La entropia de Shannon para m=10
print('La entropia de Shannon para m=10: ')
inter10,l1frec10,l1entr10 = entropia(promedio_ag,10)
print(l1entr10)